In [1]:
import instructor
from pydantic import BaseModel, Field
from openai import AzureOpenAI
from enum import Enum
from typing import List
from dotenv import load_dotenv
import os,json
import pandas as pd

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv("C:\\Users\\sourabh.mohite\\LLM_Repo\\Advanced-QA-and-RAG-Series\\Q&A-and-RAG-with-SQL-and-TabularData\\.env")

True

In [4]:
# --------------------------------------------------------------
# Regular Completion using OpenAI (with drawbacks)
# --------------------------------------------------------------

client = AzureOpenAI(
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_deployment=os.getenv("GET_DEPLOYMENT_NAME")
)

In [5]:
conversation = """{'OP':'Can I know what problem you are facing with your mobile device?', 
'CU':'My screen got damaged, it has cracks all over it.', 
'OP':'I’m sorry to hear that. How did the damage occur?', 
'CU':'I dropped it accidentally while walking.', 
'OP':'Thank you for the information. Is the device still under warranty?', 
'CU':'Yes, I purchased it just 6 months ago, so it should be covered.', 
'OP':'Great! I will guide you through the claim process. You will need to send the device for an inspection. Can I have your contact details to start the process?', 
'CU':'Sure, my name is John Doe, and my phone number is 123-456-7890.', 
'OP':'Thank you, John. I’ve noted your details. You’ll receive an email with further instructions on sending the device for inspection. Is there anything else I can assist you with today?', 
'CU':'No, that will be all for now. Thank you for your help!', 
'OP':'You’re welcome! We’ll be in touch soon. Have a great day!'}"""

In [6]:
# --------------------------------------------------------------
# Step 2: Patch your LLM with instructor
# --------------------------------------------------------------

# Instructor makes it easy to get structured data like JSON from LLMs
client = instructor.patch(client)

# --------------------------------------------------------------
# Step 3: Define Pydantic data models
# --------------------------------------------------------------

class TicketCategory(str, Enum):
    ORDER_ISSUE = "order_issue"
    ACCOUNT_ACCESS = "account_access"
    PRODUCT_INQUIRY = "product_inquiry"
    TECHNICAL_SUPPORT = "technical_support"
    BILLING = "billing"
    DEVICEDAMAGE = "device_damage"
    OTHER = "other"
    

class CustomerSentiment(str, Enum):
    ANGRY = "angry"
    FRUSTRATED = "frustrated"
    NEUTRAL = "neutral"
    SATISFIED = "satisfied"

class AgentBehavior(str, Enum):
    HELPFUL = "helpful"
    PATIENT = "patient"
    IMPATIENT = "impatient"
    AGGRESSIVE = "aggressive"
    NEUTRAL = "neutral"
    CONFIDENT = "confident"
    APOLOGETIC = "apologetic"
    PROFESSIONAL = "professional"

class TicketUrgency(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    CRITICAL = "critical"

class TicketClassification(BaseModel):
    category: TicketCategory
    urgency: TicketUrgency
    customer_sentiment: CustomerSentiment
    agent_behavior:AgentBehavior
    confidence: float = Field(ge=0, le=1, description="Confidence score for the classification")
    key_information: List[str] = Field(description="List of key points extracted from the ticket")
    suggested_action: str = Field(description="Brief suggestion for handling the ticket")
 

In [7]:
SYSTEM_PROMPT = """ You are an AI assistant for a mobile device claim management system at a Insurance company.
Your role is to analyze incoming claim calls related to mobile devices and provide structured information to help our team respond quickly and effectively.

Business Context:

We handle thousands of claims daily, including warranty claims, damage claims, technical issues, and billing disputes.
Effective categorization, prioritization, and accurate responses are key to maintaining high customer satisfaction and operational efficiency.
Claims are processed based on urgency, warranty status, agent behaviour and customer sentiment.
Your Tasks:

Categorize the claim into the most appropriate category (e.g., device damage, warranty issue, technical issue, billing dispute).
Assess the urgency of the issue (low, medium, high, critical).
Determine the customer's sentiment based on the tone of the message (positive, neutral, negative).
Determine the agent's behaviour based on the tone (helpful,patient)
Extract key information that would be helpful for the claim processing team (e.g., device model, damage description, purchase date, claim reason).
Suggest an initial action for processing the claim (e.g., inspect the device, escalate to technician, check warranty status).
Provide a confidence score for your classification based on the provided information.
Instructions:

Be objective and base your analysis solely on the information provided in the claim ticket.
If unsure about any aspect, reflect that in your confidence score.
For key_information, extract specific details like device model, damage type, purchase date, serial number, or any additional facts.
The suggested_action should be clear and actionable to facilitate quick resolution.
Remember to consider the customer sentiment and adjust your urgency assessment accordingly.
"""

In [8]:
def classify_ticket(ticket_text: str) -> TicketClassification:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        response_model=TicketClassification,
        temperature=0,
        max_retries=3,
        messages=[
            {
                "role": "system",
                "content": SYSTEM_PROMPT,
            },
            {"role": "user", "content": ticket_text}
        ]
    )
    return response

result = classify_ticket(conversation)
print(result.model_dump_json(indent=2))


{
  "category": "device_damage",
  "urgency": "medium",
  "customer_sentiment": "neutral",
  "agent_behavior": "helpful",
  "confidence": 0.9,
  "key_information": [
    "Screen damaged with cracks",
    "Device under warranty",
    "Purchased 6 months ago",
    "Contact details: John Doe, 123-456-7890"
  ],
  "suggested_action": "Send device for inspection"
}
